# Resnet Eval
This file is a helper tool to evaluate the predictions of the resnet model.
The file reads in the predictions of the resnet model, and display the images that the model predictly wrong. Make it easier to figure out the edge cases for the model

In [1]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
import IPython.display as display

tf.enable_eager_execution()

In [12]:
# Global constants
INPUT_PRED_DIR = './results.csv'
INPUT_TF_DIR = '../Mixmatch/ML_DATA/streetview_v3_64-test.tfrecord'

In [8]:
# Parse the label and image data from the tfrecord file
def parse_tf_records(file_dir):
    raw_image_dataset = tf.data.TFRecordDataset(file_dir)

    # Create a dictionary describing the features.
    image_feature_description = {
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image': tf.io.FixedLenFeature([], tf.string),
    }

    def _parse_image_function(example_proto):
      # Parse the input tf.Example proto using the dictionary above.
      return tf.io.parse_single_example(example_proto, image_feature_description)

    parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
    
    return parsed_image_dataset

In [9]:
parsed_image_dataset = parse_tf_records(INPUT_TF_DIR)

In [10]:
# Read in the predictions from the output of the resnet model. The first 2 columns are the labels, and the other 2 columns are the prediction probabilities.
results = pd.read_csv(INPUT_PRED_DIR, header=None)
labels = results.iloc[:,0:2]
pred = results.iloc[:,2:]

In [ ]:
# Enumerate through every example, and print out the wrong prediction images.
for i, image_features in enumerate(parsed_image_dataset.take(700)):
    label = str(labels.iloc[i, :].idxmax())
    prediction = str(pred.iloc[i, :].idxmax() - 2)
    
    if label != prediction:
        print("True label: " + label + "\nPred label: " + prediction)
        image_raw = image_features['image'].numpy()
        display.display(display.Image(data=image_raw))